In [2]:
import keras
import numpy as np
import tensorflow as tf
from keras.models import Sequential,load_model
import keras.backend as K
from commai_spd_chl_keras_datagen import commai_speed_chl_gen
from keras.layers import Dense, Activation,ConvLSTM2D,Lambda,GRU,BatchNormalization,Activation,Flatten,TimeDistributed,Conv2D,MaxPooling3D,MaxPooling2D,LSTM,Reshape,Dropout

Using TensorFlow backend.


In [ ]:
1*1

In [6]:
#run only once
#com_gen=commai_speed_chl_gen()
#com_gen.build_video_folder()

  0%|          | 28/20400 [00:00<01:13, 278.09it/s]

(480, 640, 3)
(32, 3, 480, 640)
(32, 3, 1)
building video Database


100%|██████████| 20400/20400 [01:35<00:00, 213.79it/s]


In [2]:
tf.config.list_physical_devices ()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
keras.backend.image_data_format()

In [7]:
seq = Sequential()

seq.add(Lambda(lambda x:tf.divide(x,255),input_shape=(3,480,640,1))) # Normalize the images between 0-1

seq.add(TimeDistributed(Conv2D(filters=16, kernel_size=(4,4), #basically "resize the images" since ConvLSTM2Ds are expensive as shit
                   padding='same')))
seq.add(TimeDistributed(MaxPooling2D(pool_size=(4,4)))) 
seq.add(Activation("relu"))

seq.add(ConvLSTM2D(filters=16, kernel_size=(8,8),
                   padding='same',return_sequences=True))
seq.add(TimeDistributed(MaxPooling2D(pool_size=(8,8)))) #pool every single dim of shared features 
seq.add(Dropout(0.3))
seq.add(BatchNormalization()) 
seq.add(Activation("relu"))


seq.add(ConvLSTM2D(filters=32,return_sequences=True, kernel_size=(8,8),
                   padding='same'))
seq.add(TimeDistributed(MaxPooling2D(pool_size=(8,8))))
seq.add(BatchNormalization())
seq.add(Activation("relu"))
seq.add(Dropout(0.2))

seq.add(Reshape((3,2*32)))

seq.add(BatchNormalization()) #better approch since data can sometimes "slip" from the normalization statistics
seq.add(GRU(50,activation="relu",return_sequences=True))

seq.add(Dense(100,))
seq.add(Activation("relu"))
seq.add(Dropout(0.1))

seq.add(Dense(1))
seq.add(Activation("relu"))

seq.compile(loss='mse', optimizer='adam')
seq.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 480, 640, 1)    0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 3, 480, 640, 16)   272       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 3, 120, 160, 16)   0         
_________________________________________________________________
activation_1 (Activation)    (None, 3, 120, 160, 16)   0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 3, 120, 160, 16)   131136    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 3, 15, 20, 16)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 15, 20, 16)    

In [10]:
batch_size=32
n_e=60
n_frames_train=18000
n_frames_test=1000
n_frames_valid=1400

train_idx,test_idx,valid_idx=commai_speed_chl_gen(n_frames_valid=n_frames_valid,
                    n_frames_test=n_frames_test,n_frames_train=n_frames_train,batch_size=batch_size).get_indexes()

load=np.load(open("idxes_21bs_GRU_18-5k.np","rb"))
train_idx=load["train_idx"]
test_idx=load["test_idx"]
valid_idx=load["valid_idx"]

train_gen=commai_speed_chl_gen(frame_idx=train_idx)
test_gen=commai_speed_chl_gen(frame_idx=test_idx)
valid_gen=commai_speed_chl_gen(frame_idx=valid_idx)

#np.savez_compressed(open("idxes_32bs_GRU_18k.np","wb+"),train_idx=train_idx,test_idx=test_idx,valid_idx=valid_idx)



(480, 640, 3)
(32, 3, 480, 640)
(32, 3, 1)
train_samples: 6000 
test_samples:333 
valid_samples: 466
(480, 640, 3)
(32, 3, 480, 640)
(32, 3, 1)
(480, 640, 3)
(32, 3, 480, 640)
(32, 3, 1)
(480, 640, 3)
(32, 3, 480, 640)
(32, 3, 1)


In [11]:
seq=load_model("new_gen_convLSTM2D-GRU-DENSEOUT-EvenMore_dropout-32bs-BatchNorm-Pool2D-8KS-TDConv2DB4_INPUTS4KS-IMNORM-07-42.39-3.13.hdf5",custom_objects={"tf":tf})
ie=7

filepath="new_gen_convLSTM2D-GRU-DENSEOUT-EvenMore_dropout-32bs-BatchNorm-Pool2D-8KS-TDConv2DB4_INPUTS4KS-IMNORM-{epoch:02d}-{val_loss:.2f}-{loss:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False)

cbs=[checkpoint]
hst=seq.fit_generator(train_gen,  max_queue_size=85,steps_per_epoch=n_frames_train/batch_size, epochs=n_e,workers=10,
        use_multiprocessing=True,validation_data=test_gen,callbacks=cbs,verbose=1,shuffle=True, initial_epoch=ie)

Epoch 8/60
563/562 [==============================] - 902s 2s/step - loss: 5.5798 - val_loss: 1.3735

Epoch 00008: saving model to new_gen_convLSTM2D-GRU-DENSEOUT-EvenMore_dropout-32bs-BatchNorm-Pool2D-8KS-TDConv2DB4_INPUTS4KS-IMNORM-08-1.37-5.58.hdf5
Epoch 9/60
563/562 [==============================] - 903s 2s/step - loss: 3.2733 - val_loss: 1.2729

Epoch 00009: saving model to new_gen_convLSTM2D-GRU-DENSEOUT-EvenMore_dropout-32bs-BatchNorm-Pool2D-8KS-TDConv2DB4_INPUTS4KS-IMNORM-09-1.27-3.27.hdf5
Epoch 10/60
563/562 [==============================] - 904s 2s/step - loss: 2.8967 - val_loss: 2.3233

Epoch 00010: saving model to new_gen_convLSTM2D-GRU-DENSEOUT-EvenMore_dropout-32bs-BatchNorm-Pool2D-8KS-TDConv2DB4_INPUTS4KS-IMNORM-10-2.32-2.90.hdf5
Epoch 11/60
563/562 [==============================] - 904s 2s/step - loss: 2.5633 - val_loss: 1.5943

Epoch 00011: saving model to new_gen_convLSTM2D-GRU-DENSEOUT-EvenMore_dropout-32bs-BatchNorm-Pool2D-8KS-TDConv2DB4_INPUTS4KS-IMNORM-11-1.59-

Process ForkPoolWorker-224:
Process ForkPoolWorker-225:
Process ForkPoolWorker-232:
Process ForkPoolWorker-221:
Process ForkPoolWorker-222:
Process ForkPoolWorker-231:
Process ForkPoolWorker-238:
Process ForkPoolWorker-240:
Process ForkPoolWorker-227:
Process ForkPoolWorker-237:
Process ForkPoolWorker-234:
Process ForkPoolWorker-239:
Process ForkPoolWorker-228:
Process ForkPoolWorker-230:
Process ForkPoolWorker-229:
Process ForkPoolWorker-233:
Process ForkPoolWorker-223:
Process ForkPoolWorker-235:
Process ForkPoolWorker-226:
Process ForkPoolWorker-236:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


In [8]:
#7 Epoches
seq=load_model("new_gen_convLSTM2D-GRU-DENSEOUT-EvenMore_dropout-32bs-BatchNorm-Pool2D-8KS-TDConv2DB4_INPUTS4KS-IMNORM-07-42.39-3.13.hdf5",custom_objects={"tf":tf})
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size, max_queue_size = 10,workers = 2,verbose=2)

3.4827985763549805

In [13]:
#11 Epoches
# this is too fucking good
seq=load_model("new_gen_convLSTM2D-GRU-DENSEOUT-EvenMore_dropout-32bs-BatchNorm-Pool2D-8KS-TDConv2DB4_INPUTS4KS-IMNORM-12-1.49-2.42.hdf5",custom_objects={"tf":tf})
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size, max_queue_size = 85,workers = 10,verbose=2)

0.7791522145271301